In [ ]:
med_combining_log_and_formation_data_1.py

In [1]:
import lasio as las
import os
import pandas as pd
import csv

In [2]:
directory = "Data/Notebook 36"

# Initialise empty list for dataframes
df_list = []

for file in os.listdir(directory):
    if file.endswith('.las'):
        f = os.path.join(directory, file)

        # Convert LAS file to a DF
        las_file = las.read(f)
        df = las_file.df()
        
        # Create a column for the Well Name
        well_name = las_file.well.WELL.value
        df['WELL'] = well_name
        
        # Make sure depth is a column rather than an index
        df = df.reset_index()
        df = df.sort_values(['WELL', 'DEPT']).reset_index(drop=True)
        df_list.append(df)

# Create a single dataframe with all wells
well_df = pd.concat(df_list)

In [3]:
well_df

,DEPT,GR,DT,RHOB,DRHO,NPHI,WELL
0,50.2002,NaN,NaN,NaN,NaN,NaN,L07-05
1,50.3002,NaN,NaN,NaN,NaN,NaN,L07-05
2,50.4002,NaN,NaN,NaN,NaN,NaN,L07-05
3,50.5002,34.355362,NaN,NaN,NaN,NaN,L07-05
4,50.6002,34.083130,NaN,NaN,NaN,NaN,L07-05
...,...,...,...,...,...,...,...
38627,3927.6000,NaN,NaN,NaN,NaN,NaN,L07-01
38628,3927.7000,NaN,NaN,NaN,NaN,NaN,L07-01
38629,3927.8000,NaN,NaN,NaN,NaN,NaN,L07-01
38630,3927.9000,NaN,NaN,NaN,NaN,NaN,L07-01


## Loading Formation Data

In [4]:
# Initialise empty list for dataframes
df_formation_list = []

for file in os.listdir(directory):
    if file.endswith('.csv'):
        f = os.path.join(directory, file)
        df = pd.read_csv(f)

        df_formation_list.append(df)

# Combine all formation data into a single dataframe
formations_df = pd.concat(df_formation_list)

In [5]:
formations_df

,Well,Stratigraphical Unit,Top,Bottom
0,L07-04,Maassluis Formation,0.00,448.00
1,L07-04,Oosterhout Formation,448.00,794.59
2,L07-04,Breda Formation,794.59,802.57
3,L07-04,Rupel Formation,802.57,890.75
4,L07-04,Asse Member,890.75,1022.00
...,...,...,...,...
36,L07-01,Ten Boer Member,3555.00,3644.00
37,L07-01,Upper Slochteren Member,3644.00,3705.00
38,L07-01,Ameland Member,3705.00,3796.00
39,L07-01,Lower Slochteren Member,3796.00,3870.50


Converting the dataframe to a dictionary

https://datascience.stackexchange.com/questions/67578/export-pandas-dataframe-to-a-nested-dictionary-from-multiple-columns

In [6]:
formations_dict = {k: f.groupby('Top')['Stratigraphical Unit'].apply(list).to_dict()
     for k, f in formations_df.groupby('Well')}

In [7]:
formations_dict

{'L07-01': {0.0: ['Upper North Sea Group'],
  697.74: ['Rupel Formation'],
  784.94: ['Asse Member'],
  834.0: ['Brussels Marl Member'],
  930.0: ['Ieper Member'],
  1310.02: ['Basal Dongen Tuffite Member'],
  1316.43: ['Landen Clay Member'],
  1332.0: ['Ommelanden Formation'],
  2161.84: ['Plenus Marl Member'],
  2163.67: ['Texel Formation'],
  2273.0: ['Upper Holland Marl Member'],
  2348.64: ['Middle Holland Claystone Member'],
  2351.58: ['Lower Holland Marl Member'],
  2354.5: ['Vlieland Claystone Formation'],
  2365.5: ['Lower Muschelkalk Member'],
  2431.0: ['Röt Claystone Member'],
  2529.0: ['Main Röt Evaporite Member'],
  2539.77: ['Solling Claystone Member'],
  2557.82: ['Basal Solling Sandstone Member'],
  2559.23: ['Hardegsen Formation'],
  2581.0: ['Detfurth Claystone Member'],
  2595.0: ['Lower Detfurth Sandstone Member'],
  2598.0: ['Volpriehausen Clay-Siltstone Member'],
  2674.0: ['Lower Volpriehausen Sandstone Member'],
  2727.0: ['Rogenstein Member'],
  2883.0: ['Ma

In [8]:
formations_dict['L07-01']

{0.0: ['Upper North Sea Group'],
 697.74: ['Rupel Formation'],
 784.94: ['Asse Member'],
 834.0: ['Brussels Marl Member'],
 930.0: ['Ieper Member'],
 1310.02: ['Basal Dongen Tuffite Member'],
 1316.43: ['Landen Clay Member'],
 1332.0: ['Ommelanden Formation'],
 2161.84: ['Plenus Marl Member'],
 2163.67: ['Texel Formation'],
 2273.0: ['Upper Holland Marl Member'],
 2348.64: ['Middle Holland Claystone Member'],
 2351.58: ['Lower Holland Marl Member'],
 2354.5: ['Vlieland Claystone Formation'],
 2365.5: ['Lower Muschelkalk Member'],
 2431.0: ['Röt Claystone Member'],
 2529.0: ['Main Röt Evaporite Member'],
 2539.77: ['Solling Claystone Member'],
 2557.82: ['Basal Solling Sandstone Member'],
 2559.23: ['Hardegsen Formation'],
 2581.0: ['Detfurth Claystone Member'],
 2595.0: ['Lower Detfurth Sandstone Member'],
 2598.0: ['Volpriehausen Clay-Siltstone Member'],
 2674.0: ['Lower Volpriehausen Sandstone Member'],
 2727.0: ['Rogenstein Member'],
 2883.0: ['Main Claystone Member'],
 3043.24: ['Z

## Merging the Formations and Well Data

In [9]:
def add_formation_name_to_df(depth, well_name):

    formations_depth = formations_dict[well_name].keys()   
    
    # Need to catch if we are at the last formation
    try:
        at_last_formation = False
        below = min([i for i in formations_depth if depth < i])
    except ValueError:
        at_last_formation = True

    # Need to catch if we are above the first listed formation
    try:
        above_first_formation = False
        above = max([i for i in formations_depth if depth > i])
    except:
        above_first_formation = True

    if above_first_formation:
        formation = ''

    # Check if the current depth matches an existing formation depth
    nearest_depth = min(formations_depth, key=lambda x:abs(x-depth))
    if depth == nearest_depth:
        formation = formations_dict[well_name][nearest_depth][0]

    else:
        if not at_last_formation:
            if depth >= above and depth <below:
                formation = formations_dict[well_name][above][0]
        else:
                formation = formations_dict[well_name][above][0]
    return formation

In [103]:
depth = 930
formations_depth = formations_dict[well_name].keys()   
print(formations_depth)

#https://stackoverflow.com/questions/36275459/find-the-closest-elements-above-and-below-a-given-number



# Need to catch if we are at the last formation
try:
    at_last_formation = False
    below = min([i for i in formations_depth if depth < i])
except ValueError:
    at_last_formation = True

# Need to catch if we are above the first listed formation
try:
    above_first_formation = False
    above = max([i for i in formations_depth if depth > i])
except:
    above_first_formation = True

if above_first_formation:
    print('')
    
# Check if the current depth matches an existing formation depth
nearest_depth = min(formations_depth, key=lambda x:abs(x-depth))
if depth == nearest_depth:
    print(f'Current Depth: {depth} is equal to {nearest_depth}')
    print(formations_dict[well_name][nearest_depth][0])

else:
    if not at_last_formation:
        if depth >= above and depth <below:
            print(f'Current Depth: {depth} is between {above} and {below}')
            print(formations_dict[well_name][above][0])
    else:
            print(f'Current Depth: {depth} is deeper than {above}')
            print(formations_dict[well_name][above][0])

dict_keys([0.0, 697.74, 784.94, 834.0, 930.0, 1310.02, 1316.43, 1332.0, 2161.84, 2163.67, 2273.0, 2348.64, 2351.58, 2354.5, 2365.5, 2431.0, 2529.0, 2539.77, 2557.82, 2559.23, 2581.0, 2595.0, 2598.0, 2674.0, 2727.0, 2883.0, 3043.24, 3068.4, 3104.5, 3105.0, 3107.0, 3514.37, 3518.06, 3525.0, 3545.0, 3554.5, 3555.0, 3644.0, 3705.0, 3796.0, 3870.5])
Current Depth: 930 is equal to 930.0
Ieper Member


In [10]:
well_df['FORMATION'] = well_df.apply(lambda x: add_formation_name_to_df(x['DEPT'], x['WELL']), axis=1)

In [11]:
well_df

,DEPT,GR,DT,RHOB,DRHO,NPHI,WELL,FORMATION
0,50.2002,NaN,NaN,NaN,NaN,NaN,L07-05,North Sea Supergroup
1,50.3002,NaN,NaN,NaN,NaN,NaN,L07-05,North Sea Supergroup
2,50.4002,NaN,NaN,NaN,NaN,NaN,L07-05,North Sea Supergroup
3,50.5002,34.355362,NaN,NaN,NaN,NaN,L07-05,North Sea Supergroup
4,50.6002,34.083130,NaN,NaN,NaN,NaN,L07-05,North Sea Supergroup
...,...,...,...,...,...,...,...,...
38627,3927.6000,NaN,NaN,NaN,NaN,NaN,L07-01,Step Graben Formation
38628,3927.7000,NaN,NaN,NaN,NaN,NaN,L07-01,Step Graben Formation
38629,3927.8000,NaN,NaN,NaN,NaN,NaN,L07-01,Step Graben Formation
38630,3927.9000,NaN,NaN,NaN,NaN,NaN,L07-01,Step Graben Formation


In [18]:
well_df.loc[(well_df['WELL'] == 'L07-01') & (well_df['DEPT'] >= 929) & (well_df['DEPT'] <= 935)]

,DEPT,GR,DT,RHOB,DRHO,NPHI,WELL,FORMATION
8641,929.0001,50.114014,154.603882,NaN,NaN,NaN,L07-01,Brussels Marl Member
8642,929.1001,50.693840,155.115875,NaN,NaN,NaN,L07-01,Brussels Marl Member
8643,929.2001,51.219246,155.709686,NaN,NaN,NaN,L07-01,Brussels Marl Member
8644,929.3001,51.917465,155.969910,NaN,NaN,NaN,L07-01,Brussels Marl Member
8645,929.4001,52.883636,155.905548,NaN,NaN,NaN,L07-01,Brussels Marl Member
8646,929.5001,53.966286,155.869080,NaN,NaN,NaN,L07-01,Brussels Marl Member
8647,929.6001,54.907707,156.127869,NaN,NaN,NaN,L07-01,Brussels Marl Member
8648,929.7001,55.735260,156.618805,NaN,NaN,NaN,L07-01,Brussels Marl Member
8649,929.8001,56.506104,157.195526,NaN,NaN,NaN,L07-01,Brussels Marl Member
8650,929.9001,56.992592,157.908783,NaN,NaN,NaN,L07-01,Brussels Marl Member
